Generate geojson file from COVID-19 cases using GADM data.

In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import re

In [88]:
df = pd.read_excel('./data/COVID-19 Philippines.xlsx')

In [74]:
df[df['Location']=='Miagao, Iloilo']

,Case No.,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Travel History,Epi_Link,Date of Onset of Symptoms,Date of Admission,...,Status,Health Status,Date of Final Status (recovered/expired),Final Diagnosis,Age Group,Location,Latitude,Longitude,Residence Lat,Residence Long
903,PH00904,F,22,Filipino,"Miagao, Iloilo",For validation,For validation,NaN,NaT,NaN,...,Admitted,For validation,NaT,NaN,21 - 30,"Miagao, Iloilo",14.615867,120.980991,10.670424,122.195


In [72]:
list(df['Location'].unique())

['Manila',
 'Taguig City',
 'Cainta, Rizal',
 'Makati City',
 'Pasig City',
 'Marikina',
 'Quezon City',
 'San Juan',
 'Pasay City',
 'Imus City, Cavite',
 'San Jose Del Monte City, Bulacan',
 'Bais City, Negros Oriental',
 'Libmanan, Camarines Sur',
 'Las Piñas',
 'Tayasan, Negros Oriental',
 'Mandaluyong',
 'Batangas City, Batangas',
 'Parañaque',
 'San Ildefonso, Bulacan',
 'Calamba City, Laguna',
 'Porac, Pampanga',
 'Orani, Bataan',
 'Lucena City, Quezon',
 'Santa Rosa City, Laguna',
 'Antipolo City, Rizal',
 'Dasmariñas City, Cavite',
 'San Fernando City, Pampanga',
 'Floridablanca, Pampanga',
 'Lipa City, Batangas',
 'Lemery, Batangas',
 'Manabo, Abra',
 'Kalookan City',
 'Malolos City, Bulacan',
 'Guiguinto, Bulacan',
 'Tarlac City, Tarlac',
 'Pantukan, Compostela Valley',
 'Silang, Cavite',
 'Muntinlupa',
 'Cotabato City, Maguindanao',
 'Baliuag, Bulacan',
 'Bacoor City, Cavite',
 'Cebu City, Cebu',
 'Taytay, Rizal',
 'Marawi City, Lanao Del Sur',
 'Tanza, Cavite',
 'Valenzuel

In [86]:
cities_gpdf = gpd.GeoDataFrame.from_file("./data/gadm36_PHL_2.shp")
barangay_case_gpd = gpd.GeoDataFrame.from_file("./data/gadm36_PHL_3.shp")

In [38]:
cities_gpdf_merged = cities_gpdf.merge(df, left_on='NAME_2', right_on='Location')

In [42]:
df_clean = df.dropna(subset=['Date of Admission'])
df_clean

,Case No.,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Travel History,Epi_Link,Date of Onset of Symptoms,Date of Admission,...,Status,Health Status,Date of Final Status (recovered/expired),Final Diagnosis,Age Group,Location,Latitude,Longitude,Residence Lat,Residence Long
0,PH00001,F,38,Chinese,None,Yes (China); Wife of PH02,China,Wife of PH2,2020-01-21,2020-01-25 00:00:00,...,Recovered,Recovered,2020-03-15,NaN,31 - 40,Manila,14.613480,120.980950,14.598727,120.982
1,PH00002,M,44,Chinese,None,Yes (China); Husband of PH1,China,Husband of PH1,2020-01-18,2020-01-25 00:00:00,...,Expired,Died,2020-02-01,Severe Pneumonia,41 - 50,Manila,14.613480,120.980950,14.598727,120.982
2,PH00003,F,60,Chinese,None,Yes (China),China,Traveled to Cebu and Bohol. Was already in Chi...,2020-01-21,2020-01-22 00:00:00,...,Recovered,Recovered,2020-03-15,NaN,51 - 60,Manila,14.615867,120.980991,14.598727,120.982
3,PH00004,M,48,Filipino,Taguig City,Yes (Japan),Japan,NaN,2020-03-03,2020-03-06 00:00:00,...,Recovered,Recovered,2020-03-19,"Asymptomatic, Negative Test Result",41 - 50,Taguig City,14.606904,121.020392,14.517274,121.05
4,PH00005,M,62,Filipino,"Cainta, Rizal",Husband of PH06,None,Husband of PH6,2020-02-25,2020-03-05 00:00:00,...,Expired,Died,2020-03-12,"Acute Respiratory Distress Syndrome, Severe pn...",61 - 70,"Cainta, Rizal",14.409635,121.037425,14.586506,121.114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,PH00141,M,63,Filipino,Parañaque,Contact of PH143,None,Contact of PH143,NaT,2020-03-16 00:00:00,...,Admitted,Asymptomatic,NaT,NaN,61 - 70,Parañaque,14.589620,121.069797,14.479309,121.019
141,PH00142,M,50,Filipino,Makati City,For validation,For validation,NaN,NaT,2020-03-16 00:00:00,...,Admitted,Asymptomatic,NaT,NaN,41 - 50,Makati City,14.615867,120.980991,14.554498,121.023
178,PH00179,M,59,Filipino,Taguig City,None,None,NaN,NaT,2020-03-11 00:00:00,...,Recovered,Recovered,NaT,"Asymptomatic, Negative Test Result",51 - 60,Taguig City,14.555719,121.048688,14.517274,121.05
200,PH00201,M,58,Filipino,"Marawi City, Lanao Del Sur",Malaysia,Malaysia,NaN,2020-03-17,2020-03-10 00:00:00,...,Expired,Died,2020-03-17,"Acute Respiratory Distress Syndrome, Severe Ac...",51 - 60,"Marawi City, Lanao Del Sur",8.004503,124.283897,8.009885,124.297


In [43]:
import numpy as np 
df_clean['Date of Admission']= df_clean['Date of Admission'].apply(lambda x: x if type(x)!=str else np.nan)
df_clean = df_clean.dropna(subset=['Date of Admission'])

/home/lpresbitero/.conda/envs/geo/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## First Case

In [7]:
case1 = df[df['Case No.'] == 'PH00001']
case1

,Case No.,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Travel History,Epi_Link,Date of Onset of Symptoms,Date of Admission,...,Status,Health Status,Date of Final Status (recovered/expired),Final Diagnosis,Age Group,Location,Latitude,Longitude,Residence Lat,Residence Long
0,PH00001,F,38,Chinese,None,Yes (China); Wife of PH02,China,Wife of PH2,2020-01-21,2020-01-25 00:00:00,...,Recovered,Recovered,2020-03-15,NaN,31 - 40,Manila,14.61348,120.98095,14.598727,120.982


In [9]:
case1_geo = cities_gpdf.merge(case1, left_on='NAME_2', right_on='Location')
case1_merged = case1_geo[['Latitude', 'Longitude']]
case1_gdf = gpd.GeoDataFrame(case1_merged, geometry=gpd.points_from_xy(case1_merged.Longitude, case1_merged.Latitude))
case1_gdf.to_file("./data/case1_gdf_ph.geojson", driver='GeoJSON')

## Third Case
Cebu City

In [66]:
case3 = pd.DataFrame({'Longitude':123.89322, 'Latitude': 10.31072}, index=[0])
case3_gdf = gpd.GeoDataFrame(case3, geometry=gpd.points_from_xy(case3.Longitude, case3.Latitude))
case3_gdf.to_file("./data/case3_gdf_cebu_ph.geojson", driver='GeoJSON')

In [67]:
case3 = pd.DataFrame({'Longitude':124.21558, 'Latitude': 9.88538}, index=[0])
case3_gdf = gpd.GeoDataFrame(case3, geometry=gpd.points_from_xy(case3.Longitude, case3.Latitude))
case3_gdf.to_file("./data/case3_gdf_bohol_ph.geojson", driver='GeoJSON')

## First Death 
February 2

In [11]:
from datetime import datetime

startdate = datetime.strptime('Jan 1 2020', '%b %d %Y')
enddate = datetime.strptime('Feb 2 2020', '%b %d %Y')

In [22]:
cities_gpdf_merged.Location.unique()

array(['San Juan', 'Kalookan City', 'Las Piñas', 'Makati City',
       'Mandaluyong', 'Manila', 'Marikina', 'Muntinlupa', 'Navotas',
       'Parañaque', 'Pasay City', 'Pasig City', 'Pateros', 'Quezon City',
       'Valenzuela'], dtype=object)

In [44]:
case2 = df_clean[df_clean['Date of Admission'] < enddate]

In [45]:
case2_geo = cities_gpdf.merge(case2, left_on='NAME_2', right_on='Location')
case2_merged = case2_geo[['Latitude', 'Longitude']]
case2_gdf = gpd.GeoDataFrame(case2_merged, geometry=gpd.points_from_xy(case2_merged.Longitude, case2_merged.Latitude))

## March 5 Mark

In [47]:
enddate_march5 = datetime.strptime('Mar 5 2020', '%b %d %Y')

In [48]:
case4 = df_clean[df_clean['Date of Admission'] < enddate_march5]
case4_geo = cities_gpdf.merge(case4, left_on='NAME_2', right_on='Location')
case4_merged = case2_geo[['Latitude', 'Longitude']]
case4_gdf = gpd.GeoDataFrame(case4_merged, geometry=gpd.points_from_xy(case4_merged.Longitude, case4_merged.Latitude))
case4_gdf_polygon = gpd.sjoin(barangay_case_gpd, case4_gdf, op='intersects')
case4_grouped = case4_gdf_polygon.groupby('NAME_3').size().reset_index(name="Cases")
case4_merged_cases = case4_gdf_polygon.merge(case4_grouped, on='NAME_3',how='left')
case4_merged_cases.to_file("./data/case4_gdf_polygon_ph.geojson", driver='GeoJSON')

/home/lpresbitero/.conda/envs/geo/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(epsg:4326 != None)
  "(%s != %s)" % (left_df.crs, right_df.crs)


As Point in Taguig

In [49]:
case4_merged = pd.DataFrame({'Longitude':121.06586, 'Latitude': 14.52810}, index=[0])
case4_gdf = gpd.GeoDataFrame(case4_merged, geometry=gpd.points_from_xy(case4_merged.Longitude, case4_merged.Latitude))
case4_gdf
case4_gdf.to_file("./data/case4_gdf_ph.geojson", driver='GeoJSON')

## First Case of Local Transmission
case 5

In [50]:
enddate_march6 = datetime.strptime('Mar 6 2020', '%b %d %Y')

In [52]:
case5 = df_clean[df_clean['Date of Admission'] < enddate_march6]
case5_geo = cities_gpdf.merge(case5, left_on='NAME_2', right_on='Location')
case5_merged = case5_geo[['Latitude', 'Longitude']]
case5_gdf = gpd.GeoDataFrame(case5_merged, geometry=gpd.points_from_xy(case5_merged.Longitude, case5_merged.Latitude))
case5_gdf_polygon = gpd.sjoin(barangay_case_gpd, case5_gdf, op='intersects')
case5_grouped = case5_gdf_polygon.groupby('NAME_3').size().reset_index(name="Cases")
case5_merged_cases = case5_gdf_polygon.merge(case5_grouped, on='NAME_3',how='left')
case5_merged_cases.to_file("./data/case5_gdf_polygon_ph.geojson", driver='GeoJSON')

/home/lpresbitero/.conda/envs/geo/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(epsg:4326 != None)
  "(%s != %s)" % (left_df.crs, right_df.crs)


As point

In [53]:
case5_merged = pd.DataFrame({'Longitude':121.12150, 'Latitude': 14.59878}, index=[0])
case5_gdf = gpd.GeoDataFrame(case5_merged, geometry=gpd.points_from_xy(case5_merged.Longitude, case5_merged.Latitude))
case5_gdf.to_file("./data/case5_gdf_ph.geojson", driver='GeoJSON')

## March 9

In [61]:
case20 = pd.DataFrame({'Longitude':120.92835, 'Latitude': 14.41464}, index=[0])
case20_gdf = gpd.GeoDataFrame(case20, geometry=gpd.points_from_xy(case20.Longitude, case20.Latitude))
case20_gdf.to_file("./data/case20_gdf_ph.geojson", driver='GeoJSON')

In [62]:
case21 = pd.DataFrame({'Longitude':121.04574, 'Latitude': 14.81408}, index=[0])
case21_gdf = gpd.GeoDataFrame(case21, geometry=gpd.points_from_xy(case21.Longitude, case21.Latitude))
case21_gdf.to_file("./data/case21_gdf_ph.geojson", driver='GeoJSON')

## March 10

In [63]:
case49 = pd.DataFrame({'Longitude':121.05254, 'Latitude': 13.75351}, index=[0])
case49_gdf = gpd.GeoDataFrame(case49, geometry=gpd.points_from_xy(case49.Longitude, case49.Latitude))
case49_gdf.to_file("./data/case49_gdf_ph.geojson", driver='GeoJSON')

## March 11

In [64]:
case39 = pd.DataFrame({'Longitude':123.00538, 'Latitude': 9.56623}, index=[0])
case39_gdf = gpd.GeoDataFrame(case39, geometry=gpd.points_from_xy(case39.Longitude, case39.Latitude))
case39_gdf.to_file("./data/case39_gdf_ph.geojson", driver='GeoJSON')

In [65]:
case40 = pd.DataFrame({'Longitude':124.39099, 'Latitude': 7.84750}, index=[0])
case40_gdf = gpd.GeoDataFrame(case40, geometry=gpd.points_from_xy(case40.Longitude, case40.Latitude))
case40_gdf.to_file("./data/case40_gdf_ph.geojson", driver='GeoJSON')

## 1000th mark
March 28

In [54]:
enddate_march28 = datetime.strptime('Mar 28 2020', '%b %d %Y')

Points

In [133]:
df_clean['Residence Lat'] = df_clean['Residence Lat'].astype(float)
df_clean['Residence Long'] = df_clean['Residence Long'].apply(lambda x: x if type(x)==float else float(re.sub(r"[,]*", "", x)))
case1k = df_clean[df_clean['Date of Admission'] < enddate_march28]
case1k_merged = case1k[['Residence Lat', 'Residence Long']]
case1k_grouped = case1k_merged.groupby(['Residence Long', 'Residence Lat']).size().reset_index(name="Cases")
case1k_gdf = gpd.GeoDataFrame(case1k_grouped, geometry=gpd.points_from_xy(case1k_grouped['Residence Long'], case1k_grouped['Residence Lat']))
case1k_gdf.to_file("./data/case1k_gdf_ph.geojson", driver='GeoJSON')

/home/lpresbitero/.conda/envs/geo/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/lpresbitero/.conda/envs/geo/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Gradient

In [57]:
# case1k_grouped_gradient = case1k_geo.groupby('NAME_2').size().reset_index(name="Cases")
case1k_grouped_gradient = gpd.GeoDataFrame(case1k, geometry=gpd.points_from_xy(now['Residence Long'], now['Residence Lat']))
case1k_merged_cases_gradient = gpd.sjoin(cities_gpdf, case1k_grouped_gradient, how="inner")
case1k_merged_cases_gradient.to_file("./data/case1k_gdf_gradient_ph.geojson", driver='GeoJSON')

## Now

In [130]:
now = df[['Residence Lat', 'Residence Long']]
now['Residence Lat'] = now['Residence Lat'].astype(float)
now['Residence Long'] = now['Residence Long'].apply(lambda x: x if type(x)==float else float(re.sub(r"[,]*", "", x)))
now_gdf = gpd.GeoDataFrame(now, geometry=gpd.points_from_xy(now['Residence Long'], now['Residence Lat']))
now_gdf_polygon = gpd.sjoin(cities_gpdf, now_gdf, how="inner")
now_grouped = now_gdf_polygon.groupby('NAME_2').size().reset_index(name="Cases")
now_merged_cases = now_gdf_polygon.merge(now_grouped, on='NAME_2',how='left')
now_merged_cases.NAME_2.unique()
# now_merged_cases.to_file("./data/now_gdf_gradient_ph.geojson", driver='GeoJSON')

/home/lpresbitero/.conda/envs/geo/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/lpresbitero/.conda/envs/geo/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/lpresbitero/.conda/envs/geo/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(epsg:4326 != None)
  "(%s != %s)" % (left_df.crs, right_df.crs)


array(['Manabo', 'Sallapadan', 'Altavas', 'Kalibo', 'Libacao', 'Malay',
       'Legazpi City', 'Balanga City', 'Limay', 'Mariveles', 'Orani',
       'Orion', 'Samal', 'Batangas City', 'Lemery', 'Lipa City', 'Lobo',
       'Nasugbu', 'Padre Garcia', 'San Pascual', 'Tanauan City',
       'Baguio City', 'La Trinidad', 'Baliuag', 'Bocaue', 'Bulacan',
       'Calumpit', 'Guiguinto', 'Malolos City', 'Marilao', 'Pulilan',
       'San Ildefonso', 'San Jose del Monte City', 'San Miguel',
       'Santa Maria', 'Gattaran', 'Piat', 'Tuao', 'Tuguegarao City',
       'Libmanan', 'Naga City', 'Pasacao', 'Jamindan', 'Mambusao',
       'Roxas City', 'Bacoor', 'Carmona', 'Cavite City', 'Dasmariñas',
       'General Mariano Alvarez', 'General Trias', 'Imus', 'Indang',
       'Kawit', 'Mendez', 'Naic', 'Rosario', 'Silang', 'Tanza', 'Ternate',
       'Cebu City', 'Lapu-Lapu City', 'Liloan', 'Mandaue City',
       'Talisay City', 'Pantukan', 'Panabo City', 'Tagum City',
       'Davao City', 'Digos City', 'M